In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

In [20]:
# Define transformations for the dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Load the MNIST training dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

# Load the MNIST test dataset
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

In [21]:
class MNIST_Model(nn.Module):
    def __init__(self):
        super(MNIST_Model, self).__init__()  # Call the parent class's constructor
        # Define layers (e.g., fully connected layers)
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)  #
    def forward(self, x):
      x = x.view(-1, 28*28)
      x = torch.relu(self.fc1(x))
      x = torch.relu(self.fc2(x))
      x = self.fc3(x)
      return x



In [22]:
model = MNIST_Model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

for epoch in range(5):  # Loop over 5 epochs
    for images, labels in train_loader:  # Loop over batches in train_loader
        optimizer.zero_grad()  # Clear gradients from the previous step
        outputs = model(images)  # Forward pass: compute model predictions
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass: compute gradients
        optimizer.step()  # Update model parameters

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


Epoch 1, Loss: 0.1546
Epoch 2, Loss: 0.2312
Epoch 3, Loss: 0.2672
Epoch 4, Loss: 0.0556
Epoch 5, Loss: 0.0305


In [23]:
#Setting the model for evaluation
model.eval()

MNIST_Model(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)

In [27]:
correct = 0
total = 0

In [28]:
#No need to calcualte gradient decnet for the testing phase so we need to disbale it
with torch.no_grad():
  for images, labels in test_loader:
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 96.74%


In [29]:
all_predictions = []

with torch.no_grad():
    for images, _ in test_loader:  # We don't need the actual labels here
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        all_predictions.extend(predicted.cpu().numpy())  # Collect predictions


In [31]:
print(all_predictions[10])

0


In [33]:
# Load the MNIST test dataset
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)
test_loader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=True)  # Batch size of 1 for single image


In [34]:
# Get a single test image and label
with torch.no_grad():  # No need to calculate gradients during inference
    for images, labels in test_loader:
        outputs = model(images)  # Forward pass: get logits (raw outputs) for this single image

        # Apply softmax to get probabilities
        probabilities = torch.softmax(outputs, dim=1)  # Convert logits to probabilities

        # Get the predicted label and the corresponding probability
        predicted_label = torch.argmax(probabilities, dim=1)  # Get the index of the highest probability

        print(f"Predicted Label: {predicted_label.item()}")
        print(f"Actual Label: {labels.item()}")

        # Print the probabilities for each class (0 to 9)
        for i in range(10):
            print(f"Class {i}: Probability = {probabilities[0][i].item():.4f}")

        break  # Just break after one image; remove this if you want to loop over the dataset

Predicted Label: 5
Actual Label: 5
Class 0: Probability = 0.0000
Class 1: Probability = 0.0000
Class 2: Probability = 0.0000
Class 3: Probability = 0.0000
Class 4: Probability = 0.0000
Class 5: Probability = 1.0000
Class 6: Probability = 0.0000
Class 7: Probability = 0.0000
Class 8: Probability = 0.0000
Class 9: Probability = 0.0000
